# Exercise 6

## SVM & Regularization


For this homework we consider a set of observations on a number of red and white wine varieties involving their chemical properties and ranking by tasters. Wine industry shows a recent growth spurt as social drinking is on the rise. The price of wine depends on a rather abstract concept of wine appreciation by wine tasters, opinion among whom may have a high degree of variability. Pricing of wine depends on such a volatile factor to some extent. Another key factor in wine certification and quality assessment is physicochemical tests which are laboratory-based and takes into account factors like acidity, pH level, presence of sugar and other chemical properties. For the wine market, it would be of interest if human quality of tasting can be related to the chemical properties of wine so that certification and quality assessment and assurance process is more controlled.

Two datasets are available of which one dataset is on red wine and have 1599 different varieties and the other is on white wine and have 4898 varieties. All wines are produced in a particular area of Portugal. Data are collected on 12 different properties of the wines one of which is Quality, based on sensory data, and the rest are on chemical properties of the wines including density, acidity, alcohol content etc. All chemical properties of wines are continuous variables. Quality is an ordinal variable with possible ranking from 1 (worst) to 10 (best). Each variety of wine is tasted by three independent tasters and the final rank assigned is the median rank given by the tasters.

A predictive model developed on this data is expected to provide guidance to vineyards regarding quality and price expected on their produce without heavy reliance on volatility of wine tasters.

In [1]:
import pandas as pd
import numpy as np

In [2]:
data_r = pd.read_csv('https://github.com/albahnsen/PracticalMachineLearningClass/raw/master/datasets/Wine_data_red.csv')
data_w = pd.read_csv('https://github.com/albahnsen/PracticalMachineLearningClass/raw/master/datasets/Wine_data_white.csv')

In [3]:
data = data_w.assign(type = 'white')

data = data.append(data_r.assign(type = 'red'), ignore_index=True)
data.sample(5)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,type
2723,7.5,0.26,0.25,1.7,0.038,29.0,129.0,0.99312,3.45,0.56,10.4,6,white
3886,6.8,0.33,0.31,7.4,0.045,34.0,143.0,0.99226,3.06,0.55,12.2,6,white
6071,7.6,0.36,0.31,1.7,0.079,26.0,65.0,0.99716,3.46,0.62,9.5,6,red
461,7.0,0.25,0.29,15.2,0.047,40.0,171.0,0.99820,3.22,0.45,9.3,5,white
1199,7.4,0.25,0.37,6.9,0.020,14.0,93.0,0.99390,3.00,0.48,10.7,7,white


# Exercise 6.1

Show the frecuency table of the quality by type of wine

In [4]:
pd.pivot_table(data,index='quality', columns='type', values='fixed acidity' , aggfunc='count')
#df2.pivot_table(values='X',rows=['Y','Z'],cols='X',aggfunc='count')

type,red,white
quality,,
3,10.0,20.0
4,53.0,163.0
5,681.0,1457.0
6,638.0,2198.0
7,199.0,880.0
8,18.0,175.0
9,NaN,5.0


# SVM

# Exercise 6.2

* Standarized the features (not the quality)
* Create a binary target for each type of wine
* Create two Linear SVM's for the white and red wines, repectively.


In [5]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,type
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6,white
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6,white
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6,white
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6,white
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6,white


In [6]:
data.columns

Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality', 'type'],
      dtype='object')

In [7]:

from sklearn import preprocessing
# Nombres de las columnas obtenidos
names = ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol']
# Scaler
scaler = preprocessing.StandardScaler()
# Hacer el .fit al df de las columnas seleccionadas
scaled_df = scaler.fit_transform(data[names])
scaled_df = pd.DataFrame(scaled_df, columns=names)
scaled_df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,-0.166089,-0.423183,0.284686,3.206929,-0.314975,0.815565,0.959976,2.102214,-1.359049,-0.546178,-1.418558
1,-0.706073,-0.240949,0.147046,-0.807837,-0.200790,-0.931107,0.287618,-0.232332,0.506915,-0.277351,-0.831615
2,0.682458,-0.362438,0.559966,0.306208,-0.172244,-0.029599,-0.331660,0.134525,0.258120,-0.613385,-0.328521
3,-0.011808,-0.666161,0.009406,0.642523,0.056126,0.928254,1.243074,0.301278,-0.177272,-0.882212,-0.496219
4,-0.011808,-0.666161,0.009406,0.642523,0.056126,0.928254,1.243074,0.301278,-0.177272,-0.882212,-0.496219


In [8]:
#####Estandarizar es normalizar para terminos del ejercicio 
##Hay que crear una nueva variable (Calidad 2)
##Luego crear dos variables binarias
##Partir la base en train y test para cada tipo de vino
##Entrenar las 2 SVMs con las bases de train

scaled_df['quality']=data['quality']
scaled_df['type']=data['type']
scaled_df['Calidad2']=np.where(scaled_df['quality']<=5,0,1)
scaled_df['White_wine']=np.where(scaled_df['type']=='white',1,0)
scaled_df['Red_wine']=np.where(scaled_df['type']=='red',1,0)
scaled_df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,type,Calidad2,White_wine,Red_wine
0,-0.166089,-0.423183,0.284686,3.206929,-0.314975,0.815565,0.959976,2.102214,-1.359049,-0.546178,-1.418558,6,white,1,1,0
1,-0.706073,-0.240949,0.147046,-0.807837,-0.200790,-0.931107,0.287618,-0.232332,0.506915,-0.277351,-0.831615,6,white,1,1,0
2,0.682458,-0.362438,0.559966,0.306208,-0.172244,-0.029599,-0.331660,0.134525,0.258120,-0.613385,-0.328521,6,white,1,1,0
3,-0.011808,-0.666161,0.009406,0.642523,0.056126,0.928254,1.243074,0.301278,-0.177272,-0.882212,-0.496219,6,white,1,1,0
4,-0.011808,-0.666161,0.009406,0.642523,0.056126,0.928254,1.243074,0.301278,-0.177272,-0.882212,-0.496219,6,white,1,1,0


In [9]:
from sklearn.model_selection import train_test_split
#df_3[df_3['pop'] > 5]

train_white, test_white = train_test_split(scaled_df[scaled_df['type']=='white'], test_size=0.3)
train_red, test_red = train_test_split(scaled_df[scaled_df['type']=='red'], test_size=0.3)
print(len(train_white))
print(len(test_white))
print(len(train_red))
print(len(test_red))
print(len(scaled_df))

3428
1470
1119
480
6497


In [10]:
#train_white, test_white = train_test_split(scaled_df, test_size=0.3)
#train_red, test_red = train_test_split(scaled_df, test_size=0.3)

In [11]:
from sklearn.svm import SVC # "Support Vector Classifier"

feature_cols = ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol']
XW = train_white[feature_cols]

yW = train_white['Calidad2']

SVM_WW = SVC(kernel='linear')
SVM_WW.fit(XW, yW)
test_white['Calidad2_pred']=SVM_WW.predict(test_white[feature_cols])

In [12]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

print('La matriz de confusión es :',confusion_matrix(test_white['Calidad2'], test_white['Calidad2_pred']))
print('El accuracy del modelo es:',accuracy_score(test_white['Calidad2'], test_white['Calidad2_pred']))

La matriz de confusión es : [[226 248]
 [110 886]]
El accuracy del modelo es: 0.7564625850340136


In [13]:
from sklearn.svm import SVC # "Support Vector Classifier"

feature_cols = ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol']
XR = train_red[feature_cols]

yR = train_red['Calidad2']

SVM_RW = SVC(kernel='linear')
SVM_RW.fit(XR, yR)
test_red['Calidad2_pred']=SVM_RW.predict(test_red[feature_cols])

In [14]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

print('La matriz de confusión es :',confusion_matrix(test_red['Calidad2'], test_red['Calidad2_pred']))
print('El accuracy del modelo es:',accuracy_score(test_red['Calidad2'], test_red['Calidad2_pred']))

La matriz de confusión es : [[171  65]
 [ 72 172]]
El accuracy del modelo es: 0.7145833333333333


# Exercise 6.3

Test the two SVM's using the different kernels (‘poly’, ‘rbf’, ‘sigmoid’)


## Con el vino blanco los diferentes kernels

In [15]:
SVM_WW = SVC(kernel='poly')
SVM_WW.fit(XW, yW)
test_white['Calidad2_pred']=SVM_WW.predict(test_white[feature_cols])
print('La matriz de confusión es :',confusion_matrix(test_white['Calidad2'], test_white['Calidad2_pred']))
print('El accuracy del modelo con kernel poly es:',accuracy_score(test_white['Calidad2'], test_white['Calidad2_pred']))

La matriz de confusión es : [[185 289]
 [ 68 928]]
El accuracy del modelo con kernel poly es: 0.7571428571428571


In [16]:
SVM_WW = SVC(kernel='rbf')
SVM_WW.fit(XW, yW)
test_white['Calidad2_pred']=SVM_WW.predict(test_white[feature_cols])
print('La matriz de confusión es :',confusion_matrix(test_white['Calidad2'], test_white['Calidad2_pred']))
print('El accuracy del modelo con kernel rbf es:',accuracy_score(test_white['Calidad2'], test_white['Calidad2_pred']))

La matriz de confusión es : [[265 209]
 [125 871]]
El accuracy del modelo con kernel rbf es: 0.7727891156462585


In [17]:
SVM_WW = SVC(kernel='sigmoid')
SVM_WW.fit(XW, yW)
test_white['Calidad2_pred']=SVM_WW.predict(test_white[feature_cols])
print('La matriz de confusión es :',confusion_matrix(test_white['Calidad2'], test_white['Calidad2_pred']))
print('El accuracy del modelo con kernel sigmoid es:',accuracy_score(test_white['Calidad2'], test_white['Calidad2_pred']))

La matriz de confusión es : [[226 248]
 [249 747]]
El accuracy del modelo con kernel sigmoid es: 0.6619047619047619


## Con el vino rojo los diferentes kernels

In [18]:
SVM_RW = SVC(kernel='poly')
SVM_RW.fit(XR, yR)
test_red['Calidad2_pred']=SVM_RW.predict(test_red[feature_cols])
print('La matriz de confusión es :',confusion_matrix(test_red['Calidad2'], test_red['Calidad2_pred']))
print('El accuracy del modelo con kernel poly es:',accuracy_score(test_red['Calidad2'], test_red['Calidad2_pred']))

La matriz de confusión es : [[176  60]
 [ 70 174]]
El accuracy del modelo con kernel poly es: 0.7291666666666666


In [19]:
SVM_RW = SVC(kernel='rbf')
SVM_RW.fit(XR, yR)
test_red['Calidad2_pred']=SVM_RW.predict(test_red[feature_cols])
print('La matriz de confusión es :',confusion_matrix(test_red['Calidad2'], test_red['Calidad2_pred']))
print('El accuracy del modelo con kernel rbf es:',accuracy_score(test_red['Calidad2'], test_red['Calidad2_pred']))

La matriz de confusión es : [[175  61]
 [ 70 174]]
El accuracy del modelo con kernel rbf es: 0.7270833333333333


In [20]:
SVM_RW = SVC(kernel='sigmoid')
SVM_RW.fit(XR, yR)
test_red['Calidad2_pred']=SVM_RW.predict(test_red[feature_cols])
print('La matriz de confusión es :',confusion_matrix(test_red['Calidad2'], test_red['Calidad2_pred']))
print('El accuracy del modelo con kernel sigmoid es:',accuracy_score(test_red['Calidad2'], test_red['Calidad2_pred']))

La matriz de confusión es : [[133 103]
 [ 87 157]]
El accuracy del modelo con kernel sigmoid es: 0.6041666666666666


La mejor SVM tomando el accuracy como métrica es en la que utilizó el vino blanco como base y con el kernel RBF

# Exercise 6.4
Using the best SVM find the parameters that gives the best performance

'C': [0.1, 1, 10, 100, 1000], 'gamma': [0.01, 0.001, 0.0001]

Se realizarán dos ciclos para recorrer todas las posibilidades de los parámetros

In [21]:
output = []
for gamma in [0.01, 0.001, 0.0001]: ####Ciclo para recorrer los diferentes parametros gamma
    for C in [0.1, 1, 10, 100, 1000]: ###Ciclo para recorrer los diferentes parametros C
        clf = SVC(C=C, gamma=gamma,kernel='rbf') ###Crear la SVM con los parametros y el kernel
        clf.fit(XW, yW)###Hacer el .fit
        test_white['Calidad2_pred']=clf.predict(test_white[feature_cols]) ##Predecir en la base test
        accuracy=accuracy_score(test_white['Calidad2'], test_white['Calidad2_pred']) ##Obtener el accuracy de la predicción
        output.append((C, gamma, accuracy)) ###Guardar el parametro C el gamma y la SVM

In [22]:
output

[(0.1, 0.01, 0.7040816326530612),
 (1, 0.01, 0.7653061224489796),
 (10, 0.01, 0.7666666666666667),
 (100, 0.01, 0.7789115646258503),
 (1000, 0.01, 0.7741496598639456),
 (0.1, 0.001, 0.6775510204081633),
 (1, 0.001, 0.7054421768707483),
 (10, 0.001, 0.7585034013605442),
 (100, 0.001, 0.7659863945578231),
 (1000, 0.001, 0.7673469387755102),
 (0.1, 0.0001, 0.6775510204081633),
 (1, 0.0001, 0.6775510204081633),
 (10, 0.0001, 0.7054421768707483),
 (100, 0.0001, 0.7585034013605442),
 (1000, 0.0001, 0.7619047619047619)]

La mejor SVM que se entrena es la que tiene los parámetros C=1000 Gamma=0.01 y que da un accuracy de 78,2%

# Exercise 6.5

Compare the results with other methods

In [23]:
#Aquí logistica
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()#solver='liblinear',C=1e9)

logreg.fit(XW, yW)
test_white['Calidad2_pred']=logreg.predict(test_white[feature_cols])
print('La matriz de confusión de la regresión logistica es :',confusion_matrix(test_white['Calidad2'], test_white['Calidad2_pred']))
print('El accuracy de la regresión logistica es:',accuracy_score(test_white['Calidad2'], test_white['Calidad2_pred']))

La matriz de confusión de la regresión logistica es : [[235 239]
 [126 870]]
El accuracy de la regresión logistica es: 0.7517006802721088


El accuracy de la regresión logística es inferior al obtenido cuando se utilizó la mejor SVM, aún así es importante tener en cuenta que para la SVM se corrieron múltiples modelos con diferentes kernel y también se buscó optimizar los parámetros de C y gamma mientras que la regresión logística sólo fue corrida una vez sin ningún tipo de variación en el solver ni ajuste

# Regularization

# Exercise 6.6


* Train a linear regression to predict wine quality (Continous)

* Analyze the coefficients

* Evaluate the RMSE

Entrenar el modelo de regresión lineal

In [24]:
import warnings
warnings.filterwarnings("ignore")

In [25]:
###Quiero predecir continuamente la calidad continua
from sklearn.linear_model import LinearRegression
train, test = train_test_split(scaled_df, test_size=0.3)

linreg = LinearRegression()

X = train[feature_cols]
y = train['quality']
linreg.fit(X, y)
test['quality_pred']=linreg.predict(test[feature_cols])

#test['quality'] = test['quality'].astype(float)

Revisar los coeficientes

In [26]:
#coef=list(linreg.coef_)
#cols=list(feature_cols)

coeficientes_lin=pd.DataFrame(np.transpose(linreg.coef_))
cols=pd.DataFrame(np.transpose(feature_cols))
coeficientes=pd.concat([cols, coeficientes_lin], axis=1, ignore_index=True)
coeficientes

,0,1
0,fixed acidity,0.103056
1,volatile acidity,-0.212806
2,citric acid,-0.007211
3,residual sugar,0.224335
4,chlorides,-0.015724
5,free sulfur dioxide,0.107133
6,total sulfur dioxide,-0.154235
7,density,-0.190775
8,pH,0.085701
9,sulphates,0.109212


Se puede inferir que las variables que las variables que más impactan el resultado son residual sugar y alcohol de manera directamente proporcional al resultado y de manera inversamente proporcional serían density y volatile acidity

Calcular el RMSE

In [27]:
from sklearn import metrics
from math import sqrt
MSE_lineal=metrics.mean_squared_error(test['quality'], test['quality_pred'])
RMSE=sqrt(MSE_lineal)
print('RMSE:', RMSE)

RMSE: 0.7435872112477134


# Exercise 6.7

* Estimate a ridge regression with alpha equals 0.1 and 1.
* Compare the coefficients with the linear regression
* Evaluate the RMSE

In [28]:
#Comparar lineal vs ridge
# alpha=0 is equivalent to linear regression
from sklearn.linear_model import Ridge
ridgereg = Ridge(alpha=0.1)
ridgereg.fit(X, y)
test['quality_pred'] = ridgereg.predict(test[feature_cols])
print('RMSE para modelo ridge con alpha 0.1:',np.sqrt(metrics.mean_squared_error(test['quality'], test['quality_pred'])))
coeficientes_rid_01=pd.DataFrame(np.transpose(ridgereg.coef_))

RMSE para modelo ridge con alpha 0.1: 0.743586492233314


In [29]:
ridgereg = Ridge(alpha=1)
ridgereg.fit(X, y)
test['quality_pred'] = ridgereg.predict(test[feature_cols])
print('RMSE para modelo ridge con alpha 1:',np.sqrt(metrics.mean_squared_error(test['quality'], test['quality_pred'])))
coeficientes_rid_1=pd.DataFrame(np.transpose(ridgereg.coef_))

RMSE para modelo ridge con alpha 1: 0.743580109213147


In [30]:
coeficientes=pd.concat([cols,coeficientes_lin, coeficientes_rid_01,coeficientes_rid_1], axis=1, ignore_index=True)
coeficientes.columns=(['Columna','Coeficiente lineal','Coeficiente Ridge 0,1','Coeficiente Ridge 0,1'])
coeficientes

,Columna,Coeficiente lineal,"Coeficiente Ridge 0,1","Coeficiente Ridge 0,1"
0,fixed acidity,0.103056,0.103011,0.102609
1,volatile acidity,-0.212806,-0.212807,-0.212818
2,citric acid,-0.007211,-0.007207,-0.007163
3,residual sugar,0.224335,0.224270,0.223691
4,chlorides,-0.015724,-0.015733,-0.015813
5,free sulfur dioxide,0.107133,0.107126,0.107057
6,total sulfur dioxide,-0.154235,-0.154218,-0.154065
7,density,-0.190775,-0.190690,-0.189932
8,pH,0.085701,0.085673,0.085426
9,sulphates,0.109212,0.109202,0.109110


Para el caso de la regresión ridge varios de los coeficientes disminuyeron considerablemente su valor como por ejemplo el de residual sugar para los que son directamente proporcionales y el de density para los que son inversamente proporcionales

El RMSE es muy similar en ambos casos de la regresión ridge de alpha 0.1 y alpha 1 al igual que con la regresión lineal

# Exercise 6.8

* Estimate a lasso regression with alpha equals 0.01, 0.1 and 1.
* Compare the coefficients with the linear regression
* Evaluate the RMSE

In [31]:
#Comparar lineal vs lasso
from sklearn.linear_model import Lasso
lassoreg = Lasso(alpha=0.01)
lassoreg.fit(X, y)
test['quality_pred'] = lassoreg.predict(test[feature_cols])

print('RMSE para modelo Lasso con alpha 0,01:',np.sqrt(metrics.mean_squared_error(test['quality'], test['quality_pred'])))
coeficientes_lasso_001=pd.DataFrame(np.transpose(lassoreg.coef_))

RMSE para modelo Lasso con alpha 0,01: 0.7443644248275202


In [32]:
lassoreg = Lasso(alpha=0.1)
lassoreg.fit(X, y)
test['quality_pred'] = lassoreg.predict(test[feature_cols])

print('RMSE para modelo Lasso con alpha 0,1:',np.sqrt(metrics.mean_squared_error(test['quality'], test['quality_pred'])))
coeficientes_lasso_01=pd.DataFrame(np.transpose(lassoreg.coef_))

RMSE para modelo Lasso con alpha 0,1: 0.7740728336413819


In [33]:
lassoreg = Lasso(alpha=1)
lassoreg.fit(X, y)
test['quality_pred'] = lassoreg.predict(test[feature_cols])

print('RMSE para modelo Lasso con alpha 1:',np.sqrt(metrics.mean_squared_error(test['quality'], test['quality_pred'])))
coeficientes_lasso_1=pd.DataFrame(np.transpose(lassoreg.coef_))

RMSE para modelo Lasso con alpha 1: 0.881476940214876


In [34]:
coeficientes_las=pd.DataFrame(np.transpose(lassoreg.coef_))
coeficientes=pd.concat([cols,coeficientes_lin, coeficientes_lasso_001,coeficientes_lasso_01,coeficientes_lasso_1], axis=1, ignore_index=True)
coeficientes.columns=(['Columna','Coeficiente lineal','Coeficiente Lasso 0,01','Coeficiente Lasso 0,1','Coeficiente Lasso 1'])
coeficientes

,Columna,Coeficiente lineal,"Coeficiente Lasso 0,01","Coeficiente Lasso 0,1",Coeficiente Lasso 1
0,fixed acidity,0.103056,0.000000,-0.000000,-0.0
1,volatile acidity,-0.212806,-0.218995,-0.115409,-0.0
2,citric acid,-0.007211,-0.000000,0.000000,0.0
3,residual sugar,0.224335,0.082030,0.000000,-0.0
4,chlorides,-0.015724,-0.016753,-0.000000,-0.0
5,free sulfur dioxide,0.107133,0.075905,0.000000,0.0
6,total sulfur dioxide,-0.154235,-0.100844,-0.000000,-0.0
7,density,-0.190775,-0.000000,-0.000000,-0.0
8,pH,0.085701,0.018459,0.000000,0.0
9,sulphates,0.109212,0.078292,0.000000,0.0


Los coeficientes de la regresión de Lasso son cada vez más pequeños al incrementar el valor de alpha, esto hace también que el RMSE sea cada vez mayor lo que quiere decir que las predicciones son menos cada vez menos ajustadas a la realidad

# Exercise 6.9

* Create a binary target

* Train a logistic regression to predict wine quality (binary)

* Analyze the coefficients

* Evaluate the f1score

In [35]:
#El binary target es el mismo del principio
#Aquí logistica
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()#solver='liblinear',C=1e9)

X = train[feature_cols]
y = train['Calidad2']

logreg.fit(X, y)
test['Calidad2_pred']=logreg.predict(test[feature_cols])
print('La matriz de confusión de la regresión logistica es :',confusion_matrix(test['Calidad2'], test['Calidad2_pred']))
print('El accuracy de la regresión logistica es:',accuracy_score(test['Calidad2'], test['Calidad2_pred']))

La matriz de confusión de la regresión logistica es : [[ 417  318]
 [ 193 1022]]
El accuracy de la regresión logistica es: 0.737948717948718


In [36]:
coeficientes_log=pd.DataFrame(np.transpose(logreg.coef_))
cols=pd.DataFrame(np.transpose(feature_cols))
coeficientes_log=pd.concat([cols, coeficientes_log], axis=1, ignore_index=True)
coeficientes_log

,0,1
0,fixed acidity,0.125376
1,volatile acidity,-0.731992
2,citric acid,-0.050533
3,residual sugar,0.425001
4,chlorides,-0.043206
5,free sulfur dioxide,0.285432
6,total sulfur dioxide,-0.455986
7,density,-0.183590
8,pH,0.158919
9,sulphates,0.307096


El coeficiente de la variable alcohol es mucho más grande que con las otras regresiones

In [37]:
from sklearn.metrics import f1_score
print('El F1SCORE de la regresión logistica es:',f1_score(test['Calidad2'], test['Calidad2_pred']))

El F1SCORE de la regresión logistica es: 0.8


# Exercise 6.10

* Estimate a regularized logistic regression using:
* C = 0.01, 0.1 & 1.0
* penalty = ['l1, 'l2']
* Compare the coefficients and the f1score

In [38]:
X = train[feature_cols]
y = train['Calidad2']

output = []
coeficientes_1 = cols##[]
colnames_1 = []
for C in [0.01, 0.1, 1]: ####Ciclo para recorrer los diferentes parametros c
    for penalty in ['l1', 'l2']: ###Ciclo para recorrer las diferentes penalidades
        logreg = LogisticRegression(C=C, penalty=penalty,solver='liblinear')
        logreg.fit(X, y)###Hacer el .fit
        test['Calidad2_pred']=logreg.predict(test[feature_cols]) ##Predecir en la base test
        f1=f1_score(test['Calidad2'], test['Calidad2_pred']) ##Obtener el f1score de la predicción
        output.append((C, penalty, f1)) ###Guardar
        coeficientes_tmp=pd.DataFrame(np.transpose(logreg.coef_))
        coeficientes_1=np.column_stack([coeficientes_1, coeficientes_tmp])

In [39]:
output

[(0.01, 'l1', 0.7875443437130468),
 (0.01, 'l2', 0.7962456003128666),
 (0.1, 'l1', 0.7967097532314924),
 (0.1, 'l2', 0.7974931453192323),
 (1, 'l1', 0.8),
 (1, 'l2', 0.8)]

In [40]:
coeficientes_1=pd.DataFrame(coeficientes_1)
coeficientes_1.columns=(['Columna','C0.01-l1','C0.01-l2','C0.1-l1','C0.1-l2','C1-l1','C1-l2'])
coeficientes_1

,Columna,C0.01-l1,C0.01-l2,C0.1-l1,C0.1-l2,C1-l1,C1-l2
0,fixed acidity,0,0.0714325,0.0112451,0.116414,0.106023,0.125376
1,volatile acidity,-0.487509,-0.571524,-0.716833,-0.711714,-0.73547,-0.731992
2,citric acid,0,0.00319195,-0.0224336,-0.0431244,-0.0485249,-0.0505331
3,residual sugar,0.0184101,0.284498,0.282273,0.404412,0.400593,0.425001
4,chlorides,0,-0.0804598,-0.042682,-0.0486987,-0.0436126,-0.0432062
5,free sulfur dioxide,0,0.187351,0.24123,0.271713,0.282346,0.285432
6,total sulfur dioxide,0,-0.315986,-0.395642,-0.436671,-0.450444,-0.455986
7,density,0,-0.164238,0,-0.177536,-0.145791,-0.18359
8,pH,0,0.117753,0.0906646,0.152575,0.147411,0.158919
9,sulphates,0.108705,0.251638,0.267951,0.300177,0.301681,0.307096


Disminuir el valor de C hace que algunas de las variables dejen de tener importancia si se utiliza la penalidad l1, para el caso de la penalidad l2 se tiene que si bien las variables tienen coeficientes menores para valores de c mas pequeños estos no se vuelven cero como en el caso de la penalidad l1

En el caso del f1Score el mayor se presenta con c=0.1 y penalidad =l1, aún así las variaciones en el f1score no son tan grandes al hacer modificaciones en el modelo